In [10]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from subprocess import Popen, PIPE
import os
import pathlib
import shutil

In [14]:
path_genome = '/home/juan/Desktop/juan/bio/data/genomes/rice/IRGSP-1.0_genome.fasta'
path_blast = 'data/wheat/allmites.fasta.csv'
organism = 'rice'
params = {'min_len':50,'max_len':False,'min_distance':5,'max_q':1.25,'min_q':0.85,'min_pident':96,'min_qcov':93}

In [2]:
path_genome = '/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.dna.toplevel.fa'
path_gff = 'data/wheat/allmites.filtered.gff'
organism = 'wheat'

In [ ]:
path_out_dir = 'data/' + organism + '/by_family/'

In [15]:
#read blast output
df = pd.read_csv(path_blast, sep='\t', header=None)
df.columns = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
print('initial:',len(df.index))
initial = len(df.index)

initial: 36523233


In [5]:
df = pd.read_csv(path_gff, sep='\t')

In [6]:
df.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
print(len(df.index))

22129373


In [7]:
fasta_seq = SeqIO.parse(path_genome, 'fasta')
buffer_mites = {}

In [ ]:
for record in fasta_seq:
    dff_extract = df[df.seqname == record.id]
    count = 0
    curr = 0
    total = len(dff_extract.index)
    print(record.id, len(dff_extract.index))
    clean_seq = ''.join(str(record.seq).splitlines())
    for k,v in dff_extract.iterrows():
        start = min(v.start,v.end)
        end = max(v.start,v.end)
        new_seq = clean_seq[start:end]
        att = v.attribute
        id = att + "_" + record.id + "_" + str(start) + '_' + str(end)
        seq = SeqRecord(Seq(new_seq), id=id, description="_")
        if not att in buffer_mites:
            buffer_mites[att] = []
        buffer_mites[att].append(seq)
        count += 1
        curr_new = int(count * 100 * 1.0 / (total * 1.0))
        if curr_new != curr:
            curr = curr_new
            if curr_new % 1 == 0:
                print(curr_new)